<a href="https://colab.research.google.com/github/JibKh/NVIDIA-FlowNet2-Google-Colab/blob/master/FlowNet2_Colab_RAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Install FlowNet2

In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Mon Oct 19 20:17:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+

In [2]:
import os
!git clone https://github.com/princeton-vl/RAFT.git
os.chdir("RAFT")

Cloning into 'RAFT'...
remote: Enumerating objects: 129, done.
remote: Total 129 (delta 0), reused 0 (delta 0), pack-reused 129
Receiving objects: 100% (129/129), 10.00 MiB | 10.32 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
!pip install flowiz -U

In [20]:
!./download_models.sh

# User Input and Restart

In [112]:
# FOR INFERENCE
video_frames = 2 # If you have a video put 1. If you have frames put 2.

frames_zip_name = "1 - Video_Export.zip" # If you have the frames, enter its zip file here. For ex: "3 - Video.zip"
frames_directory = '../gdrive/My Drive/Hajj Videos/Frames/' # Where the frames are located. If its in gdrive: '../gdrive/My Drive/Location of Zip/' Change the Location of Zip to wherever yours is stored.
no_frames_skip = None # How many frames you want skipped. This works in modulus manner. If you input 1, all the frames are skipped. 3 means every third frame is skipped etc. Leave at None to not skip frames.
no_average_frames = None # How many frames you want to average.
flow_video_name = frames_zip_name[0:-4] + '.mp4'

video_name = None # If you have a video you want to run inference on. Please include .mp4 or whatever extension the video has.
video_local_gdrive = None # If you want to upload a video from your local drive, choose 1. If from your google drive, choose 2. If some other option, go to section "Upload Video".
video_gdrive = None # File_id for your google drive video. Use this link to see how to get file ID https://docs.meiro.io/books/meiro-integrations/page/where-can-i-find-the-file-id-on-google-drive#:~:text=To%20locate%20the%20File%20ID,%3D%60%20is%20the%20File%20ID.

In [113]:
# If you are using Google Drive to access frames, mount it here. The first time you run it, it will give you a prompt.
if video_frames == 2:
  from google.colab import drive
  drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [114]:
# If you are uploading a video from your local machine, please do so here.
if video_frames == 1 and video_local_gdrive == 1:
  from google.colab import files
  uploaded = files.upload()
  if video_name != list(uploaded.keys())[0]:
    video_name = list(uploaded.keys())[0]

In [115]:
# To restart and run again. Change any parameters above and then go click the "User Input and Restart" cell. Then Runtime -> Run After.
!rm -r ./frames
!rm -r ./Flo
!rm -r ./FlowFrames
!rm -r ./output
!rm ./$video_name
!rm -r ./FlowVideo
!rm -r ./Backup
!rm -r ./FlowImages
!rm -r ./FlowFrames2
!rm -r ./FlowVideo.mp4

rm: cannot remove './frames': No such file or directory
rm: cannot remove './Flo': No such file or directory
rm: cannot remove './FlowFrames': No such file or directory
rm: cannot remove './output': No such file or directory
rm: cannot remove './None': No such file or directory
rm: cannot remove './FlowVideo': No such file or directory
rm: cannot remove './Backup': No such file or directory
rm: cannot remove './FlowImages': No such file or directory
rm: cannot remove './FlowFrames2': No such file or directory
rm: cannot remove './FlowVideo.mp4': No such file or directory


# Run the inference

## Setup Video

### Upload Video

1) Upload your own video from Google Drive. <br>
2) Upload a video from your local machine. They will be saved in flownet2pytorch folder.

In [116]:
# Download from Google drive
if video_frames == 1 and video_local_gdrive == 2:
  from google_drive_downloader import GoogleDriveDownloader as gdd
  gdd.download_file_from_google_drive(file_id=video_gdrive, dest_path=video_name)

In [117]:
# # Or upload from local machine
# if video_frames == 1 and video_local_gdrive == 1:
#   from google.colab import files
#   uploaded = files.upload()
#   if video_name != list(uploaded.keys())[0]:
#     video_name = list(uploaded.keys())[0]

### Converting video to frames

In [118]:
if video_frames == 1:
  import os
  def mkdir_ifnotexists(dir):
      if os.path.exists(dir):
          return
      os.mkdir(dir)

In [119]:
if video_frames == 1:
  vid_file = video_name
  frame_pth = './frames'
  mkdir_ifnotexists(frame_pth)
  cmd = "ffmpeg -i %s -start_number 0 -vsync 0 %s/frame_%%06d.png" % (
              vid_file,
              frame_pth,
          )
  os.system(cmd)

## Setup Frames

### Download Frames

In [120]:
# if video_frames == 2:
#   !mkdir -p ./frames

#   from google.colab import drive
#   drive.mount('/content/gdrive')

In [121]:
if video_frames == 2:
  !mkdir -p ./frames
  unzip_file = frames_directory + frames_zip_name
  !unzip '$unzip_file' -d ./frames

Archive:  ../gdrive/My Drive/Hajj Videos/Frames/1 - Video_Export.zip
mapname:  conversion of  failed
 extracting: ./frames/20180820231746378_UTC+01.jpg  
 extracting: ./frames/20180820231829711_UTC+01.jpg  
 extracting: ./frames/20180820231835045_UTC+01.jpg  
 extracting: ./frames/20180820232010247_UTC+01.jpg  
 extracting: ./frames/20180820231919045_UTC+01.jpg  
 extracting: ./frames/20180820232012113_UTC+01.jpg  
 extracting: ./frames/20180820231839111_UTC+01.jpg  
 extracting: ./frames/20180820231758379_UTC+01.jpg  
 extracting: ./frames/20180820231909777_UTC+01.jpg  
 extracting: ./frames/20180820231949847_UTC+01.jpg  
 extracting: ./frames/20180820231843044_UTC+01.jpg  
 extracting: ./frames/20180820231842044_UTC+01.jpg  
 extracting: ./frames/20180820231938246_UTC+01.jpg  
 extracting: ./frames/20180820231802313_UTC+01.jpg  
 extracting: ./frames/20180820231823579_UTC+01.jpg  
 extracting: ./frames/20180820231800113_UTC+01.jpg  
 extracting: ./frames/20180820231913377_UTC+01.jpg 

### Rename Frames

In [122]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

2262


In [123]:
if video_frames == 2:
  import os

  file_dir = "./frames/"
  for count, filename in enumerate(sorted(os.listdir(file_dir))):
    # print(filename)
    if filename[-11:] == "_UTC+01.jpg":
      src = file_dir + filename
      dst = file_dir + str(count).zfill(6) + '.png'
      os.rename(src, dst)

In [124]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

2262


### Skip Frames

In [125]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

2262


In [126]:
if no_frames_skip != None:
  directory = './frames'
  # no_frames_skip = 2

  for i, file in enumerate(sorted(os.listdir(directory))):
    if (file[-4:] == ".png") and (int(file[0:-4]) % no_frames_skip == 0):
      # print(file)
      os.remove(directory+'/' + file)

In [127]:
if no_frames_skip != None:
  import os

  file_dir = "./frames/"
  for count, filename in enumerate(sorted(os.listdir(file_dir))):
    # print(filename)
    if filename[-4:] == ".png":
      src = file_dir + filename
      dst = file_dir + str(count).zfill(6) + '.png'
      os.rename(src, dst)

In [128]:
!ls ./frames | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.

2262


## Run Inference

In [129]:
import time

start_time = time.time()
print(start_time)

!mkdir ./FlowFrames
!python demo.py --model=models/raft-things.pth --path=frames

1603140515.1878119
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
2

# Testing Average

In [130]:
if no_average_frames != None:

  import numpy as np
  from pathlib import Path
  import os
  from utils.flow_utils import writeFlow

  def write_frame(flow, i):
    dir = "./Flo/" + str(i).zfill(6) + ".flo"
    writeFlow(dir, flow)

  def average_list(list1):
    length_list = len(list1)
    temp_addition = 0
    for i in range(0, length_list):
        temp_addition += list1[i]
    temp_val = temp_addition / length_list
    return temp_val
    # try:
    #   return (list1[0] + list1[1]) / 2
    # except:
    #   print("Error in averaging")

  flow_list = [None] * no_average_frames
  index_flow = 0
  index_name = 0

  !mkdir ./Flo
  dir = './output/inference/run.epoch-0-flow-field/'

  for i, flo_file in enumerate(sorted(os.listdir(dir))):
    if flo_file[-3:] != "flo":
      continue

    path = Path(dir + flo_file)
    with path.open(mode='r') as flo:
      tag = np.fromfile(flo, np.float32, count=1)[0]
      width = np.fromfile(flo, np.int32, count=1)[0]
      height = np.fromfile(flo, np.int32, count=1)[0]
      nbands = 2
      tmp = np.fromfile(flo, np.float32, count= nbands * width * height)
      flow = np.resize(tmp, (int(height), int(width), int(nbands)))

      # Method 1
      # if i == 0:
      #   flow_list[0] = flow
      # else:
      #   flow_list[1] = flow
      #   average_flow = average_list(flow_list)
      #   write_frame(average_flow, i-1)
      #   flow_list[0] = average_flow

      # Method 2
      # if index_flow == 0:
      #   flow_list[index_flow] = flow
      #   index_flow += 1
      # else:
      #   flow_list[index_flow] = flow
      #   index_flow = 0
      #   average_flow = average_list(flow_list)
      #   write_frame(average_flow, index_name)
      #   index_name += 1

      # Method 3
      if (index_flow % no_average_frames == 0) and index_flow != 0:
        average_flow = average_list(flow_list)
        write_frame(average_flow, index_name)
        index_flow = 0
        index_name += 1

      flow_list[index_flow] = flow
      index_flow += 1

      if i == len(os.listdir(dir)) - 1:
        average_flow = average_list(flow_list)
        write_frame(average_flow, index_name)

      os.remove(dir + flo_file)

In [131]:
# !ls ./Flo | wc -l # Use to recheck if number of frames is consistent and nothing went wrong.
# # !rm -r ./Flo

In [132]:
# !mkdir ./Flo

# from utils.flow_utils import writeFlow

# for i in range(0, len(new_flows)):
#   dir = "./Flo/" + str(i).zfill(6) + ".flo"
#   writeFlow(dir, new_flows[i])

# Flowiz technique

In [133]:
# !python -m flowiz \
# ./FlowFrames/*.flo \
# -o FlowImages \
# -v FlowVideo \
# -r 10

# !mv ./FlowVideo/000000.flo.mp4 './FlowVideo/$flow_video_name'

In [134]:
# from google.colab import files
# files.download('./FlowVideo/'+flow_video_name)

# Visualizing flo files

### Install scipy as some tensorflow functionality requires updated scipy

In [135]:
!pip install scipy==1.4.1

### Define show_flow() for visualization.
 Original Source https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py

In [136]:
# Source:https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py
import matplotlib.pyplot as plt
import numpy as np

UNKNOWN_FLOW_THRESH = 1e7
def show_flow(filename):
    """
    visualize optical flow map using matplotlib
    :param filename: optical flow file
    :return: None
    """
    flow = read_flow(filename)
    img = flow_to_image(flow)
    plt.imshow(img)
    plt.show()

def read_flow(filename):
    """
    read optical flow from Middlebury .flo file
    :param filename: name of the flow file
    :return: optical flow data in matrix
    """
    f = open(filename, 'rb')
    magic = np.fromfile(f, np.float32, count=1)
    data2d = None

    if 202021.25 != magic:
        print ('Magic number incorrect. Invalid .flo file')
    else:
        w = int(np.fromfile(f, np.int32, count=1)[0])
        h = int(np.fromfile(f, np.int32, count=1)[0])
        #print("Reading %d x %d flo file" % (h, w))
        data2d = np.fromfile(f, np.float32, count=2 * w * h)
        # reshape data into 3D array (columns, rows, channels)
        data2d = np.resize(data2d, (h, w, 2))
    f.close()
    return data2d

def flow_to_image(flow):
    """
    Convert flow into middlebury color code image
    :param flow: optical flow map
    :return: optical flow image in middlebury color
    """
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    maxu = -999.
    maxv = -999.
    minu = 999.
    minv = 999.

    idxUnknow = (abs(u) > UNKNOWN_FLOW_THRESH) | (abs(v) > UNKNOWN_FLOW_THRESH)
    u[idxUnknow] = 0
    v[idxUnknow] = 0

    maxu = max(maxu, np.max(u))
    minu = min(minu, np.min(u))

    maxv = max(maxv, np.max(v))
    minv = min(minv, np.min(v))

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(-1, np.max(rad))

    #print( "max flow: %.4f\nflow range:\nu = %.3f .. %.3f\nv = %.3f .. %.3f" % (maxrad, minu,maxu, minv, maxv))

    u = u/(maxrad + np.finfo(float).eps)
    v = v/(maxrad + np.finfo(float).eps)

    img = compute_color(u, v)

    idx = np.repeat(idxUnknow[:, :, np.newaxis], 3, axis=2)
    img[idx] = 0

    return np.uint8(img)


def compute_color(u, v):
    """
    compute optical flow color map
    :param u: optical flow horizontal map
    :param v: optical flow vertical map
    :return: optical flow in color code
    """
    [h, w] = u.shape
    img = np.zeros([h, w, 3])
    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    colorwheel = make_color_wheel()
    ncols = np.size(colorwheel, 0)

    rad = np.sqrt(u**2+v**2)

    a = np.arctan2(-v, -u) / np.pi

    fk = (a+1) / 2 * (ncols - 1) + 1

    k0 = np.floor(fk).astype(int)

    k1 = k0 + 1
    k1[k1 == ncols+1] = 1
    f = fk - k0

    for i in range(0, np.size(colorwheel,1)):
        tmp = colorwheel[:, i]
        col0 = tmp[k0-1] / 255
        col1 = tmp[k1-1] / 255
        col = (1-f) * col0 + f * col1

        idx = rad <= 1
        col[idx] = 1-rad[idx]*(1-col[idx])
        notidx = np.logical_not(idx)

        col[notidx] *= 0.75
        img[:, :, i] = np.uint8(np.floor(255 * col*(1-nanIdx)))

    return img


def make_color_wheel():
    """
    Generate color wheel according Middlebury color code
    :return: Color wheel
    """
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255*np.arange(0, RY) / RY))
    col += RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.transpose(np.floor(255*np.arange(0, YG) / YG))
    colorwheel[col:col+YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.transpose(np.floor(255*np.arange(0, GC) / GC))
    col += GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.transpose(np.floor(255*np.arange(0, CB) / CB))
    colorwheel[col:col+CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.transpose(np.floor(255*np.arange(0, BM) / BM))
    col += + BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / MR))
    colorwheel[col:col+MR, 0] = 255

    return colorwheel

### Visualize

In [137]:
# show_flow('./Flo/000001.flo')

### Save Flo files as images

In [138]:
import os
import PIL.Image
def mkdir_ifnotexists(dir):
    if os.path.exists(dir):
        return
    os.mkdir(dir)


# flo_pth='/content/flownet2pytorch/output/inference/run.epoch-0-flow-field/'
flo_pth = './FlowFrames/'
flos=[flo_pth + f for f in sorted(os.listdir(flo_pth))]
mkdir_ifnotexists('./FlowImages')
length = len(flos)
for i in range(length):
  if flos[i][-3:] == "flo":
    print(i+1, "/", length, flos[i])
    PIL.Image.fromarray(flow_to_image(read_flow(flos[i]))).save('./FlowImages/'+os.path.basename(flos[i])+'.png')
    os.remove(flos[i])

1 / 2261 ./FlowFrames/000000.flo
2 / 2261 ./FlowFrames/000001.flo
3 / 2261 ./FlowFrames/000002.flo
4 / 2261 ./FlowFrames/000003.flo
5 / 2261 ./FlowFrames/000004.flo
6 / 2261 ./FlowFrames/000005.flo
7 / 2261 ./FlowFrames/000006.flo
8 / 2261 ./FlowFrames/000007.flo
9 / 2261 ./FlowFrames/000008.flo
10 / 2261 ./FlowFrames/000009.flo
11 / 2261 ./FlowFrames/000010.flo
12 / 2261 ./FlowFrames/000011.flo
13 / 2261 ./FlowFrames/000012.flo
14 / 2261 ./FlowFrames/000013.flo
15 / 2261 ./FlowFrames/000014.flo
16 / 2261 ./FlowFrames/000015.flo
17 / 2261 ./FlowFrames/000016.flo
18 / 2261 ./FlowFrames/000017.flo
19 / 2261 ./FlowFrames/000018.flo
20 / 2261 ./FlowFrames/000019.flo
21 / 2261 ./FlowFrames/000020.flo
22 / 2261 ./FlowFrames/000021.flo
23 / 2261 ./FlowFrames/000022.flo
24 / 2261 ./FlowFrames/000023.flo
25 / 2261 ./FlowFrames/000024.flo
26 / 2261 ./FlowFrames/000025.flo
27 / 2261 ./FlowFrames/000026.flo
28 / 2261 ./FlowFrames/000027.flo
29 / 2261 ./FlowFrames/000028.flo
30 / 2261 ./FlowFrames/

### Generate video from Flo

In [139]:
os.system('ffmpeg -r 25 -i FlowImages/%6d.flo.png -vcodec libx264 -b 10M -y FlowVideo.mp4')

print("My program took", time.time() - start_time, "to run")
print(time.time())

My program took 6312.8925840854645 to run
1603146828.0808415


In [140]:
from google.colab import files
files.download('FlowVideo.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [141]:
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('FlowVideo.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML("""
# <video width=400 controls>
#       <source src="%s" type="video/mp4">
# </video>
# """ % data_url)